## Test

In [ ]:
%env SESSION=abcdef1234abc
%env USERID=123456
%env LEADERBOARD=123456

In [1]:
from datetime import date, datetime
from textwrap import dedent
import requests
import json
import pytz
import svg
import os

In [ ]:
session = os.environ.get('SESSION')
if session is None or session == '':
    print('No session found!')
    raise SystemExit(1)
userid = os.environ.get('USERID')
if userid is None or userid == '':
    print('No userid found!')
    raise SystemExit(1)
leaderboard = os.environ.get('LEADERBOARD')
if leaderboard is None or leaderboard == '':
    print('No leaderboard found - but continuing anyway!')
file = os.environ.get('FILE')
if file is None or file == '':
    print('No file found - but continuing anyway!')

year = int(date.today().year)

if leaderboard is None or leaderboard == '':
    leaderboard = f'https://adventofcode.com/{year}/leaderboard/private/view/{userid}.json'
else:
    leaderboard = f'https://adventofcode.com/{year}/leaderboard/private/view/{leaderboard}.json'

print('Session:', session)
print('Userid:', userid)
print('Leaderboard:', leaderboard)
print('File:', file)
print('Year:', year)

In [ ]:
cookie = {'session': session}
print('Fetching leaderboard data: ' + leaderboard)
r = requests.get(leaderboard, cookies=cookie)
if r.status_code != 200:
    print(f'Got status code {r.status_code}: {r.text}')
    raise SystemExit(1)
try:
    data = json.loads(r.text)
except json.JSONDecodeError as err:
    print('Could not parse json from leaderboard. Check the leaderboard id and the session cookie.')
    print(err)
    raise SystemExit(1)

In [ ]:
user = data['members'][userid]
stars = user['stars']
score = user['local_score']
member_scores = [member['local_score'] for member in data['members'].values()]
member_scores.sort(reverse=True)
position = member_scores.index(score) + 1

days_completed = 0
for day in data['members'][userid]['completion_day_level']:
    if '2' in data['members'][userid]['completion_day_level'][day]:
        days_completed += 1

new_york_tz = pytz.timezone('America/New_York')
today = datetime.now(new_york_tz).date()

if today < datetime(year, 12, 1, tzinfo=new_york_tz).date():
    day = 0
elif today > datetime(year, 12, 31, tzinfo=new_york_tz).date():
    day = 24
else:
    day = today.day

print(f'Day: {day}')
print(f'Days completed: {days_completed}')
print(f'Stars: {stars}')
print(f'Score: {score}')
print(f'Position: {position}')

In [4]:
day = 4
days_completed = 3
stars = 6
score = 301
position = 2
leaderboard = 276352

In [14]:
elements = [
    svg.Rect(
        x=0, y=0,
        rx=5, ry=5,
        width=100, height=40,
        fill='#22272e'
    ),
    svg.G(
        id='logo'
    ),
    svg.Text(
        x=45, y=12,
        font_size=6,
        fill='#cdd9e5',
        text=f'Days: {days_completed}/{day}',
        font_family='Consolas, monospace'
    ),
    svg.Text(
        x=45, y=22,
        font_size=6,
        fill='#cdd9e5',
        text=f'Stars: {stars}',
        font_family='Consolas, monospace'
    ),
    svg.Text(
        x=77, y=22,
        font_size=6,
        fill='#ffff3e',
        text='*',
        font_family='Consolas, monospace'
    ),
]

if leaderboard is not None:
    elements.append(
        svg.Text(
            x=45, y=32,
            font_size=6,
            fill='#cdd9e5',
            text=f'Score: {score} ({position})'
        )
    )

canvas = svg.SVG(
    width=100,
    height=40,
    elements=elements
)

canvas_parts = str(canvas).split('<g id="logo"/>')

with open('./aoc-logo.svg', 'r') as f:
    logo = f.read()

canvas_with_logo = canvas_parts[0] + logo + canvas_parts[1]
canvas_with_logo = dedent(canvas_with_logo).replace('\n', '').strip()

with open('./test.svg', 'w') as f:
    f.write(canvas_with_logo)

In [ ]:
with open('README.md', 'r') as file:
    readme_content = file.readlines()

start_marker = "<!-- START_AOC_BADGE -->\n"
end_marker = "<!-- END_AOC_BADGE -->\n"

start_index = readme_content.index(start_marker) + 1
end_index = readme_content.index(end_marker)

if today < datetime(year, 11, 15, tzinfo=new_york_tz).date() and today > datetime(year, 1, 15, tzinfo=new_york_tz).date():
    print("Too early this year. Removing AoC badge.")
    new_readme_content = readme_content[:start_index] + readme_content[end_index:]
else:
    new_readme_content = readme_content[:start_index] + [dedent("""
    <a href="https://adventofcode.com/">
      <img width="50%" alt="AoC Badge" src="./test.svg"/>
    </a>
""")] + readme_content[end_index:]

with open('README.md', 'w') as file:
    file.writelines(new_readme_content)